# Controlling a Cart-Pendulum System

In [ ]:
from scipy.integrate import solve_ivp
from scipy.signal import place_poles

import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate

from cart_pendulum import CartPend

## Setup (DO NOT CHANGE THESE PARAMETERS)

The cell below sets up the parameters for the mass, length, and damping ratio.

In [ ]:
m = 1
M = 5
L = 2
g = 10
b = 1

A = np.array(
  [[0, 1, 0, 0],
   [0, -b/M, -m*g/M, 0],
   [0, 0, 0, 1],
   [0, b/(M*L), (m+M)*g/(M*L), 0]])

B = np.reshape([0, 1/M, 0, 1/(M*L)], (4, 1))

cart_pend = CartPend(m, M, L, g, b)

## Part (c)

Show that the linearized pendulum-cart system is controllable but unstable. You can either work it out by hand or fill in the cells below. 

The functions `np.column_stack` and `np.linalg.eig` may be helpful.

In [ ]:
##### Workspace #####
C = 
print('The rank of the controllabilty matrix is {}.'.format(np.linalg.matrix_rank(C)))

eigs = 
print('The eigenvalues of the state matrix are {}.'.format(eigs))

Let's see what happens if we push the pendulum very slightly by giving input $u = 0.01$

In [ ]:
def exponential_decay(t,x):
  u = [0.01]
  return A.dot(x) + B.dot(u)

sol = solve_ivp(exponential_decay, (0, 10), (0, 0, 0, 0), t_eval=np.linspace(0,10,300))
CartPend.plot_response(sol)

<b>In your written submission, explain what happens when the cart-pendulum when a small force input is applied.</b>

## Part (d)

Since the system is controllable, we can use state-feedback to place the eigenvalues anywhere in the complex plane. 

<b> Find the values of K by hand or by filling in the cells below to make the eigenvalues of the closed-loop matrix equal to [-2, -1.4, -1.2, -1.3] </b>  

When implementing this in practice, we can use the `place_poles` function from the scipy.signal library: https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.place_poles.html

In [ ]:
## YOUR CODE HERE ##

eigs = 
K = 

Now that we have picked the eigenvalues of our system, let's see how the linearized system behaves for $t \in [0, 10].$

In [ ]:
def linearized_response(t,x):
  u = -K.dot(x)
  return A.dot(x) + B.dot(u)

sol = solve_ivp(linearized_response, (0, 10), (0, 0, 0.1, 0), t_eval=np.linspace(0,10,300))
CartPend.plot_response(sol)

As a sanity check, let's make sure the nonlinear system behaves in a similar fashion.

In [ ]:
def nonlinear_response(t, x):
  u = -K.dot(x)
  sin = np.sin(x[2])
  cos = np.cos(x[2])
  D = m*L*L*(M+m*sin**2)

  f1 = x[1]
  f2 = (1/D)*(-m**2*L**2*g*sin*cos + m*L*L*(m*L*x[3]**2*sin - b*x[1])) + m*L*L*(1/D)*u
  f3 = x[3]
  f4 = (1/D)*((m+M)*m*g*L*sin - m*L*cos*(m*L*x[3]**2*sin - b*x[1])) + m*L*cos*(1/D)*u + 0.05*np.random.normal(0,1)
  return f1,f2,f3,f4

sol = solve_ivp(nonlinear_response, (0, 10), (0, 0, 0 + 0.1, 0), t_eval=np.linspace(0,10,200))
CartPend.plot_response(sol)

## Part (f) ##

### Applying a reference input $\vec{r}$

In part (e) we showed that if our target $\vec{r} \in \mbox{Nul}(A)$ then the error $\vec{e}(t)$ converges to $\vec{0}$ meaning the state $\vec{x}$ will reach the target $\vec{r}.$

Now let's try to move our cart-pendulum system from $\vec{0}$ to the state $\vec{r} = \begin{bmatrix} 1 \\ 0 \\ 0 \\ 0 \end{bmatrix}.$ 

Since $\vec{r} \in \mbox{Nul}(A)$ we can apply an input $u = -K (\vec{x} - \vec{r})$ to drive the steady-state error to zero.

In [ ]:
r = np.array([1, 0, 0, 0])

The plots of the linearized response are shown below.

In [ ]:
n = 200
start, end = 0, 10
x0 = np.zeros(4)

sol = cart_pend.response(x0, start, end, n, K=K, r=r, linear=True)
CartPend.plot_response(sol)

### Eigenvalue Placement ###

Since the linearized system is a good approximation of the nonlinear cart-pendulum system, we can take the following steps.
1. Pick our favorite choice of eigenvalues $\lambda_{1}$ to $\lambda_{4},$ for the linearized system.
2. Find the appropriate $K$ values to place the eigenvalues of the closed-loop matrix $A_{cl} = (A - BK)$ at $\lambda_{i}.$
3. Apply the input $u = K(\vec{r} - \vec{x})$ to the original nonlinear system. 

Try tweaking the eigenvalues of the system and running the simulation to see how the cart-pendulum system behaves. 
Note that the eigenvalues need to be distinct for `place_poles` to solve for $K.$

You may also need to install ffmpeg to play the simulation video

<font color='red'><b>Warning: If the values of $K$ become too large the differential equation solver will break.<b></font>

In [ ]:
from IPython.display import HTML

# Values are filled in as a reference. Try to keep your eigenvalues distinct.
eigs = [-4, -3, -1+4j, -1-4j] ## CHANGE THESE VALUES

K = place_poles(A, B, eigs).gain_matrix
print('Feedback gain matrix K = {}'.format(K))

# Code to generate response plots. Feel free to change start, end, and n.
start, end, n = 0, 10, 200
sol = cart_pend.response(x0, start, end, n, K=K, r=r, linear=False)
CartPend.plot_response(sol)

In [ ]:
# You can change the length of the simulation and ylim sets the x-axis of the plot.
anim = cart_pend.simulate(K, r, start=0.0, end=8.0, ylim=(-1, 2) ,n=n); 

# May require installing ffmpeg 
HTML(anim.to_html5_video())

<b> In your written submission, explain the effect that the eigenvalues have on the system. </b> What was your favorite pair of eigenvalues?

## Final Remarks ###

Congratulations on reaching the end of this homework problem. Hopefully you were able to see the end-to-end process of analyzing and controlling a system by taking the following steps:
1. Linearizing the nonlinear system around an equilibrium point.
2. Determining whether the system was controllable.
3. Realizing the system was unstable.
4. Using feedback control to stabilize the system and move it to a desired target.

Since we simulated the cart-pendulum system, there were still some nonidealities such as noise and saturation that we didn't take into account. When implementing this system in the real world, we won't always get the freedom of picking arbitrarily large K values. 

If you take EE128, you will get to implement the cart-pendulum system in the lab! On the other hand, if you take EE127, you can learn about ways to pick K values that have the least "cost" by using a technique called the Linear Quadratic Regulator or LQR.